# PRODIGY RECIPES

#### Idea: This notebook is to automate all the recipe generations including the paths and names of the datasets. 

#### Author: Vaishnavi Kandala

#### Date: 7th Jan 2022

#### Note: Each recipe components are rightly used before running the recipe. Ensure the right PORT is used.

In [1]:
import pandas as pd
import re

In [2]:
# this is the reference to all the categories in the dataset - currently we have around 73 categories.
# path = "/Users/vaishnavikandala/Desktop/Personal/Climate-Scanner/trends_&_innovation_classes.tsv"
# df_categories = pd.read_csv(path,sep="\t")
# df_categories

In [2]:
def category_name_processing(category_name):
    category_name = category_name.lower()
    category_name = re.sub(" ","_",category_name)
    category_name = re.sub("/","_",category_name)
    category_name = re.sub("-","_",category_name)
    return category_name

category_name_processing("Capsule wardrobe")

'capsule_wardrobe'

## General CONFIG

In [3]:
log_path = "/home/azureuser/christine/logs/"
patterns_path = "/home/azureuser/christine/patterns/"
raw_data_path = "/home/azureuser/christine/data/"
model_path = "/home/azureuser/christine/model/"

In [13]:
### GENERAL CONFIG
category_name = "artificial intelligence"
category_name_processed = category_name_processing(category_name)

### TASK1
port_value_task1 = str(8084)
task1_name = category_name_processed + "_keywords"
seed_words = "'artificial intelligence, machine learning, natural language processing, supervised learning, deep learning'"
task1_log = log_path + task1_name + ".out"


### TASK2
port_value_task2 = str(8084)
task2_name = category_name_processed + "_sentences"
task2_data_path = raw_data_path + category_name_processed + "_raw_data.jsonl"
task2_log = log_path + task2_name + ".out"


### TASK3
port_value_task3 = str(8094)


In [14]:
task2_data_path

'/home/azureuser/christine/data/artificial_intelligence_raw_data.jsonl'

PORTS ASSIGNED

PORT = 8084 path = http://51.132.188.89:8083/
PORT = 8086 path = http://51.132.188.89:8085/


### RECIPE 1

In [9]:
### TASK 1 - RECIPE 1


# Recipe 1 - This is mainly to get suggestions on the keywords relevant to a particular category. We are using a pre-trained model to get these suggestions (since, our categories are generic). We think we have good suggestions with this model but it can be further explored.

# PRODIGY_PORT=8084 PRODIGY_CONFIG_OVERRIDES='{"validate": false}' nohup prodigy sense2vec.teach AI_Seed_Words model/s2v_old --seeds "artificial intelligence, machine learning, natural language processing, supervised learning, deep learning" > AIseedsOut.out 2>&1 &

# Description:

# PRODIGY_PORT - please use the specific port assigned to your task.
# PRODIGY_CONFIG_OVERRIDES - to override any generic settings in the configuration. Here we do validate - false to ensure no input is missed because of formats.
# sense2vec.tech - is the prodigy model or recipe
# AI_Seed_Words - name of the dataset
# models/s2v_old - is the selected model to generate embeddings.
# seeds - seed words for the category
# output location - AISeedsOut.out is the name of the output (log) file.

In [15]:

### RECIPE Generation

"PRODIGY_PORT=" + port_value_task1  + \
" PRODIGY_CONFIG_OVERRIDES='{\'validate\': false}'" + \
" nohup prodigy sense2vec.teach " + task1_name + \
" /home/azureuser/christine/model/s2v_reddit_2019_lg --seeds " + seed_words + " > " + task1_log + " 2>&1 &"


"PRODIGY_PORT=8084 PRODIGY_CONFIG_OVERRIDES='{'validate': false}' nohup prodigy sense2vec.teach artificial_intelligence_keywords /home/azureuser/christine/model/s2v_old --seeds 'artificial intelligence, machine learning, natural language processing, supervised learning, deep learning' > /home/azureuser/christine/logs/artificial_intelligence_keywords.out 2>&1 &"

#### GENERATE PATTERNS

In [16]:
### CONVERT SEED WORDS TO PATTERNS
task1_patterns = patterns_path + task1_name + ".jsonl"

"prodigy terms.to-patterns " +  task1_name + " " + task1_patterns + " --label " + category_name_processed

'prodigy terms.to-patterns artificial_intelligence_keywords /home/azureuser/christine/patterns/artificial_intelligence_keywords.jsonl --label artificial_intelligence'

### RECIPE 2

In [ ]:
# Recipe 2 - This is mainly to train an initial model by selecting the sentences with the keywords (generated in the previous recipe). The more exhaustive keywords are a better model can be trained.

#PRODIGY_PORT=8084 nohup prodigy textcat.teach AI_Words en_core_web_sm prodigyData/AI_TrainingData.jsonl --label "Artificial Intelligence" --patterns prodigyData/AI_Seed_Word_Patterns.jsonl > AIwordsOut.out 2>&1 &

# Description:

# PRODIGY_PORT - use the specific port for the task.
# textcat.teach - prodigy recipe to train a base model
# AI_Words - the name of the dataset
# en_core_web_sm - model for embeddings
# AI_TrainingData.jsonl - training data formatted in JSONL format (use the GitHub function to generate prodigy formatted dataset).
# label - label for this call.
# patterns - keywords generated using recipe 1 to further select specific category based sentences.

In [17]:
### RECIPE GENERATION
"PRODIGY_PORT=" + port_value_task2  + \
" PRODIGY_CONFIG_OVERRIDES='{\'feed_overlap\': false}'" + \
" nohup prodigy textcat.teach " +  task2_name + \
" en_core_web_sm "+ task2_data_path + \
" --label " + category_name_processed + \
" --patterns " + task1_patterns + \
" --exclude " + task2_name + \
" > " + task2_log + " 2>&1 &"

"PRODIGY_PORT=8084 PRODIGY_CONFIG_OVERRIDES='{'feed_overlap': false}' nohup prodigy textcat.teach artificial_intelligence_sentences en_core_web_sm /home/azureuser/christine/data/artificial_intelligence_raw_data.jsonl --label artificial_intelligence --patterns /home/azureuser/christine/patterns/artificial_intelligence_keywords.jsonl --exclude artificial_intelligence_sentences > /home/azureuser/christine/logs/artificial_intelligence_sentences.out 2>&1 &"

### RECIPE 3

In [ ]:
# Recipe 3 - This is mainly to retrain the model for that category using the annotations gathered using recipe 2.

#PRODIGY_PORT=8084 prodigy train --textcat-multilabel AI_Words --label-stats --verbose --eval-split 0.2 model/AI-model

# Description:

# PRODIGY_PORT - use specific port for the task.
# textcat_multilabel - retraining the model with the annotation data.
# AI_Words - dataset used for annotations in recipe 2.
# eval_split - 80% train 20% test.
# AI-model - model used.
# This recipe will give the performance Precision / Recall / F-Score for the class.

In [20]:
task3_model_path = model_path + category_name_processed + "_model"

"prodigy train --textcat-multilabel " + \
task2_name + " --label-stats --verbose --eval-split 0.2 " + \
task3_model_path


'prodigy train --textcat-multilabel artificial_intelligence_sentences --label-stats --verbose --eval-split 0.2 /home/azureuser/christine/model/artificial_intelligence_model'